# Data setup

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

C:\Users\taraf\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\taraf\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\taraf\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\taraf\anaconda3\envs\tensorflow\lib\site-packages

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import os

In [5]:
os.listdir('/Datasets/cell_images/cell_images')

['Parasitized', 'Uninfected']

In [6]:
pos_list = os.listdir('/Datasets/cell_images/cell_images/Parasitized')

In [7]:
neg_list = os.listdir('/Datasets/cell_images/cell_images/Uninfected')

In [8]:
print(len(pos_list))
print(len(neg_list))

13780
13780


In [10]:
base = '/Datasets/base'

In [18]:
os.mkdir('/Datasets/base')

In [19]:
train='/Datasets/base/train'
os.mkdir(train)

In [20]:
valid = '/Datasets/base/valid'
os.mkdir(valid)

In [21]:
train_pos = '/Datasets/base/train/Parasitized'
train_neg  = '/Datasets/base/train/Uninfected'
os.mkdir(train_pos)
os.mkdir(train_neg)

In [22]:
valid_pos=('/Datasets/base/valid/Parasitized')
valid_neg=('/Datasets/base/valid/Uninfected')
os.mkdir(valid_pos)
os.mkdir(valid_neg)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
pos_train, pos_valid = train_test_split(pos_list,test_size=0.3)

In [25]:
neg_train, neg_valid = train_test_split(neg_list,test_size=0.3)

In [26]:
print(len(pos_train))
print(len(pos_valid))
print(len(neg_train))
print(len(neg_valid))

9646
4134
9646
4134


In [27]:
from shutil import copyfile

In [28]:
def label_data(train_list,valid_list,DEST_train,DEST_valid,name):
    for i in train_list:
        copyfile(os.path.join(name,i),DEST_train + '/' + i)    
    for j in valid_list:
        copyfile(os.path.join(name,j),DEST_valid+ '/' + j)    

In [29]:
name1 = '/Datasets/cell_images/cell_images/Parasitized'
name2  = '/Datasets/cell_images/cell_images/Uninfected'

In [30]:
pos = label_data(pos_train,pos_valid,train_pos,valid_pos,name1)

In [31]:
neg = label_data(neg_train,neg_valid,train_neg,valid_neg,name2)

In [32]:
os.listdir(valid_pos)[0:2]

['C100P61ThinF_IMG_20150918_144104_cell_164.png',
 'C100P61ThinF_IMG_20150918_144104_cell_167.png']

# TF Model 

In [33]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0,rotation_range=40,height_shift_range=0.2,
                                   width_shift_range=0.2,shear_range=0.2,zoom_range=0.2,
                                 horizontal_flip=True,fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [34]:
train_gen = train_datagen.flow_from_directory(train,target_size =(300,300),batch_size=64,class_mode='binary')
valid_gen = valid_datagen.flow_from_directory(valid,target_size =(300,300),batch_size=64,class_mode='binary')

Found 19291 images belonging to 2 classes.
Found 8267 images belonging to 2 classes.


In [35]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape=(300,300,3),include_top=False,weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.
16277504/87910968 [====>.........................] - ETA: 31:21

KeyboardInterrupt: 

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output
last_layer.output_shape

(None, 17, 17, 768)

In [36]:
# model creation
activation='relu'

model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Conv2D(64,(5,5),activation=activation,input_shape=(300,300,3)))
model1.add(tf.keras.layers.MaxPooling2D(2,2))
#2nd CNN
model1.add(tf.keras.layers.Conv2D(32,(5,5),activation=activation))
model1.add(tf.keras.layers.MaxPooling2D(2,2))
#3rd CNN
model1.add(tf.keras.layers.Conv2D(16,(5,5),activation=activation))
model1.add(tf.keras.layers.MaxPooling2D(2,2))
#4th CNN
model1.add(tf.keras.layers.Conv2D(8,(5,5),activation=activation))
model1.add(tf.keras.layers.MaxPooling2D(2,2))

model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(152,activation=activation))
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(152,activation=activation))
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(152,activation=activation))
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [37]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_94 (Conv2D)           (None, 296, 296, 64)      4864      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 148, 148, 64)      0         
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 144, 144, 32)      51232     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 72, 72, 32)        0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 68, 68, 16)        12816     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 34, 34, 16)        0         
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 30, 30, 8)         3208      
__________

In [38]:
optimizer='adam'
loss = 'binary_crossentropy'
metrics=['accuracy']
model1.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [39]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('accuracy')>0.92):
            print('Achieved 92% accuracy')
            self.model.stop_training=True
callback = mycallback()
    

In [40]:
history = model1.fit(train_gen,epochs=20,verbose=1,validation_data=valid_gen,steps_per_epoch=80,callbacks=[callback],use_multiprocessing=False,workers=16)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
  6/302 [..............................] - ETA: 2:29:24 - loss: 1.0008 - acc: 0.5312

KeyboardInterrupt: 